# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

# this is where we write code in fragments. then this finalized code will be put into etl.py
1.from song_data json files populate song and artist tables
2.from log_data jsonson filesles populate user,time tablesles
3. use both tables above to create song_plays table

ETL--> json files--> postgres tables

In [37]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import json

In [38]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=password")
cur = conn.cursor()

In [39]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [40]:
song_files = get_files("data/song_data/")
len(song_files)

71

In [41]:
filepath = song_files[0]

In [42]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [43]:
df.values[0].tolist()

['ARD7TVE1187B99BFB1',
 nan,
 'California - LA',
 nan,
 'Casual',
 218.93179,
 1,
 'SOMZWCG12A8C13C480',
 "I Didn't Mean To",
 0]

In [44]:
song_data = df[['song_id', 'title','artist_id', 'year', 'duration']].values[0].tolist()

In [45]:
song_data = (song_data[0], song_data[1], song_data[2], song_data[3], song_data[4])
song_data

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [46]:
song_table_insert

' INSERT INTO songs(song_id,title,artist_id,year,duration) values(%s,%s,%s,%s,%s) ON CONFLICT(song_id) DO NOTHING'

In [47]:

cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [48]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data = (artist_data[0], artist_data[1], artist_data[2], artist_data[3], artist_data[4])

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [49]:
artist_table_insert

'INSERT INTO  artists(artist_id, name,location,lattitude, longitude)\n                          VALUES(%s, %s, %s, %s, %s)\n                          ON CONFLICT (artist_id) \n                          DO NOTHING'

In [50]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [51]:
log_files = get_files("data/log_data/")

In [52]:
filepath = log_files[0]
filepath

'C:\\Users\\dnyanadapramod.arjun\\Desktop\\UDACITY DATA ENGINEER\\project1\\data\\log_data\\2018\\11\\2018-11-01-events.json'

In [53]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [54]:
df = df[df['page']=="NextSong"].reset_index()
df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
1,4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [55]:
import datetime
t = pd.to_datetime(df.ts, unit='ms')
t.head()

0   2018-11-01 21:01:46.796
1   2018-11-01 21:05:52.796
2   2018-11-01 21:08:16.796
3   2018-11-01 21:11:13.796
4   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [56]:
df['week'] = t.apply(lambda x: datetime.date(x.year, x.month, x.day).isocalendar()[1])
df['week_day'] = t.apply(lambda x: datetime.date(x.year, x.month, x.day).strftime("%A"))
df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,...,page,registration,sessionId,song,status,ts,userAgent,userId,week,week_day
0,2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",...,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,44,Thursday
1,4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",...,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,44,Thursday
2,5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",...,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,44,Thursday
3,6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",...,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,44,Thursday
4,7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",...,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,44,Thursday


In [57]:
time_df.head(3)

,day,hour,month,start_time,week,weekday,year
0,1,21,11,2018-11-01 21:01:46.796,44,Thursday,2018
1,1,21,11,2018-11-01 21:05:52.796,44,Thursday,2018
2,1,21,11,2018-11-01 21:08:16.796,44,Thursday,2018


In [58]:
time_data = (t, t.dt.hour, t.dt.day, df.week, t.dt.month, t.dt.year, df.week_day)
column_labels = ['start_time','hour','day','week','month', 'year','weekday']
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()
df['start_time'] = t
df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,...,registration,sessionId,song,status,ts,userAgent,userId,week,week_day,start_time
0,2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",...,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,44,Thursday,2018-11-01 21:01:46.796
1,4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",...,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,44,Thursday,2018-11-01 21:05:52.796
2,5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",...,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,44,Thursday,2018-11-01 21:08:16.796
3,6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",...,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,44,Thursday,2018-11-01 21:11:13.796
4,7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",...,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,44,Thursday,2018-11-01 21:17:33.796


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [59]:
time_df_in_order=time_df[['start_time','hour','day','week','month','year','weekday']]
time_df_in_order.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,Thursday
1,2018-11-01 21:05:52.796,21,1,44,11,2018,Thursday
2,2018-11-01 21:08:16.796,21,1,44,11,2018,Thursday
3,2018-11-01 21:11:13.796,21,1,44,11,2018,Thursday
4,2018-11-01 21:17:33.796,21,1,44,11,2018,Thursday


In [60]:
time_table_insert

'INSERT INTO time(start_time,hour,day,week,month, year,weekday)\n                        VALUES(%s, %s, %s, %s, %s, %s, %s)\n                        ON CONFLICT (start_time) \n                        DO NOTHING'

In [61]:
for i, row in time_df_in_order.iterrows():
    #print(list(row))
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [62]:
user_df = df[['userId','firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,8,Kaylee,Summers,F,free
1,8,Kaylee,Summers,F,free
2,8,Kaylee,Summers,F,free
3,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [63]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [30]:
song_select="""SELECT songs.song_id, artists.artist_id FROM songs JOIN artists ON  songs.artist_id=artists.artist_id WHERE songs.title=%s AND artists.name=%s AND songs.duration=%s;"""

In [31]:
songplay_table_insert

'INSERT INTO song_plays( start_time, user_id,level,artist_id,song_id, session_id, location, user_agent)\n                            VALUES(%s, %s, %s, %s, %s, %s, %s, %s)'

In [35]:
for index, row in df.iterrows():
    
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
#     print(results)
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
    # insert songplay record
    songplay_data = (str(row.start_time),row.userId,row.level,songid,artistid, row.sessionId,row.location,row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [64]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.